In this assignment, you will use a pre-trained convnet to produce features for a classifier that can detect a single object type. This notebook has some code to help you get started. 

In [2]:
import pandas as pd
import cv2
import os
from os import listdir
from os.path import isfile, join
import os.path as osp
from tqdm import tqdm_notebook as tqdm

### Gather positive examples

Pick a word. For example, "red" or "santa" or "horse". 

Now you will need to find "positive" image examples of that word. For example, if you chose "red" as your word, you will need to find images of red things. You are free to use Google Image search or something similar. File types shouldn't matter, but try to stick with .png and .jpg files.

You'll need at least 100 positive example images. Put them in the folder called `pos`. 

### Gather negative examples

Now you need to think about negative examples; i.e., things that are *not* examples of your word. You can either just find random images, or look for specific negative examples. For example, if you chose the word "red" then it might work best if you find negative examples that are other colors, especially colors close to red. 

You'll need at least 200 negative example images. Put them in the folder called `neg`. 

## 1.) Run the following cell

* This imports needed Keras libraries
* Then, it gets the trained VGG19 imagenet model
* Then, it prints out the names of all the layers in that model

In [3]:
import numpy as np
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

base_model = VGG19(weights='imagenet',include_top=True)
xs,ys=224,224

for layer in base_model.layers:
    if 'conv' not in layer.name:
        continue
    filters,biases = layer.get_weights()
    print(layer.name, filters.shape)
for layer in base_model.layers:
    print(layer.name)

block1_conv1 (3, 3, 3, 64)
block1_conv2 (3, 3, 64, 64)
block2_conv1 (3, 3, 64, 128)
block2_conv2 (3, 3, 128, 128)
block3_conv1 (3, 3, 128, 256)
block3_conv2 (3, 3, 256, 256)
block3_conv3 (3, 3, 256, 256)
block3_conv4 (3, 3, 256, 256)
block4_conv1 (3, 3, 256, 512)
block4_conv2 (3, 3, 512, 512)
block4_conv3 (3, 3, 512, 512)
block4_conv4 (3, 3, 512, 512)
block5_conv1 (3, 3, 512, 512)
block5_conv2 (3, 3, 512, 512)
block5_conv3 (3, 3, 512, 512)
block5_conv4 (3, 3, 512, 512)
input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


### 2.) Determine your output layer

- try `predictions` first

In [4]:
layer = 'predictions'

model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Run the following cell

- These functions are to help you perform transfer learning

In [6]:
def get_image(img_path, xs,ys):
    x = image.load_img(img_path, target_size=(xs, ys))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

def get_img_features(model, img):
    img = preprocess_input(img)
    yhat = model.predict(img)
    return yhat

def get_image_features(word):
    files = [f for f in listdir(word)] # grab all of the images in the folder
    image_vectors = []
    for f in tqdm(files):
        img = get_image(osp.join(word, f), xs, ys) 
        x_feats = get_img_features(model, img).flatten() # get features for each image
        image_vectors.append(x_feats) 
    return np.array(image_vectors)

## 3.) Evaluate a classifier for your `word`

* Using the positive and negative output from `model`, train a classifier (it can be a linear classifier from scikit-learn, if you'd like, but I would recommend the Keras Dense network we built for the previous assignment). 
* You'll need to split your data into Train and Test (I would recommend using half of the data for training, half for testing; you may opt for downloading more positive and negative examples)
* your classifier can be any scikit classifier, but you can also use a neural network of some kind

In [7]:
pos_images = get_image_features('pos') # get positive image vectors
neg_images = get_image_features('neg') # get negative image vectors

<ipython-input-6-729984638232>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f in tqdm(files):


  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
len(pos_images)

102

### Prepare the data. Split to train/test sets

#### chair train and test

In [9]:
chair_train_set = pos_images[:len(pos_images)//2]
len(chair_train_set)

51

In [10]:
len(chair_train_set[0])

1000

In [11]:
chair_test_set = pos_images[len(pos_images)//2:]
len(chair_test_set)

51

#### non_chari train and test

In [12]:
non_chair_train_set = neg_images[:len(neg_images)//2]
len(non_chair_train_set)

100

In [85]:
non_chair_test_set = neg_images[len(neg_images)//2:]
len(non_chair_test_set)

100

In [86]:
# 1 for chairs and 0 for non_chairs
SAMPLE_SIZE_chair = 51
SAMPLE_SIZE_nonChair = 100
# generate X and Y (inputs and labels).
x_train = np.concatenate([chair_train_set, non_chair_train_set])
labels_train = np.asarray([1 for _ in range(SAMPLE_SIZE_chair)]+[0 for _ in range(SAMPLE_SIZE_nonChair)])
labels_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [87]:
x_valid = np.concatenate([chair_test_set, non_chair_test_set])
labels_valid = np.asarray([1 for _ in range(SAMPLE_SIZE_chair)]+[0 for _ in range(SAMPLE_SIZE_nonChair)])

In [61]:
non_chair_test_set

array([[1.48303852e-05, 7.88832913e-05, 1.74738143e-05, ...,
        6.99300244e-06, 9.30785518e-06, 2.71459023e-04],
       [4.54932831e-07, 1.04534263e-06, 4.05560570e-07, ...,
        2.37108424e-07, 5.02159446e-06, 1.21944242e-04],
       [7.57037697e-07, 1.54355803e-07, 1.12764241e-07, ...,
        1.05624069e-08, 1.17557511e-06, 1.03811044e-06],
       ...,
       [2.23305915e-06, 7.31057080e-05, 2.84153248e-05, ...,
        3.75375566e-06, 3.92634756e-05, 1.69469378e-04],
       [5.02819285e-06, 1.18010124e-04, 3.48515532e-05, ...,
        7.19825891e-07, 4.18537056e-05, 5.80088142e-03],
       [1.38154951e-06, 4.85703140e-07, 4.31130644e-07, ...,
        1.71531511e-07, 2.25817603e-05, 4.91841580e-04]], dtype=float32)

In [96]:
x_valid

array([[8.52906908e-07, 1.52429504e-06, 1.20660761e-05, ...,
        1.18848379e-06, 1.41164019e-05, 3.28911774e-05],
       [1.20390752e-07, 4.92091665e-08, 7.45863701e-07, ...,
        1.69185239e-06, 1.57996499e-06, 3.58630379e-04],
       [4.55534170e-08, 3.12140465e-08, 4.06423428e-09, ...,
        3.24686994e-07, 1.23909194e-06, 3.95332318e-05],
       ...,
       [2.23305915e-06, 7.31057080e-05, 2.84153248e-05, ...,
        3.75375566e-06, 3.92634756e-05, 1.69469378e-04],
       [5.02819285e-06, 1.18010124e-04, 3.48515532e-05, ...,
        7.19825891e-07, 4.18537056e-05, 5.80088142e-03],
       [1.38154951e-06, 4.85703140e-07, 4.31130644e-07, ...,
        1.71531511e-07, 2.25817603e-05, 4.91841580e-04]], dtype=float32)

In [84]:
#import tensorflow as tf

In [83]:
#arr = tf.one_hot([[0,51],[1,100]], 2).numpy()
#len(arr)

In [76]:
#labels_train = np.asarray(labels_train).astype('float32').reshape((-1,1))
#labels_valid = np.asarray(labels_valid).astype('float32').reshape((-1,1))

In [88]:
labels_valid

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [89]:
labels_train

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [18]:
x_train.shape

(151, 1000)

In [19]:
each_pixel = 1000

In [20]:
labels_train.shape

(151,)

In [97]:
x_valid.shape

(151, 1000)

### Define model, train

In [115]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Flatten, Dense
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.layers import Input

In [116]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [114]:
model = Sequential()
model.add(Dense(1024, input_dim=1000, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [91]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 1025      
Total params: 2,075,649
Trainable params: 2,075,649
Non-trainable params: 0
_________________________________________________________________


In [92]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [93]:
history = model.fit(x_train, labels_train, epochs=20, batch_size=64, validation_data=(x_valid, labels_valid))

Epoch 1/20
3/3 [==============================] - 2s 192ms/step - loss: 0.6737 - accuracy: 0.6752 - val_loss: 0.5983 - val_accuracy: 0.8013
Epoch 2/20
3/3 [==============================] - 0s 120ms/step - loss: 0.5658 - accuracy: 0.8056 - val_loss: 0.4873 - val_accuracy: 0.8146
Epoch 3/20
3/3 [==============================] - 0s 119ms/step - loss: 0.4436 - accuracy: 0.8194 - val_loss: 0.3757 - val_accuracy: 0.8146
Epoch 4/20
3/3 [==============================] - 0s 76ms/step - loss: 0.3327 - accuracy: 0.8065 - val_loss: 0.2817 - val_accuracy: 0.8675
Epoch 5/20
3/3 [==============================] - 0s 63ms/step - loss: 0.2186 - accuracy: 0.9147 - val_loss: 0.2014 - val_accuracy: 0.9470
Epoch 6/20
3/3 [==============================] - 0s 62ms/step - loss: 0.1345 - accuracy: 0.9750 - val_loss: 0.1498 - val_accuracy: 0.9669
Epoch 7/20
3/3 [==============================] - 0s 72ms/step - loss: 0.0869 - accuracy: 0.9914 - val_loss: 0.1243 - val_accuracy: 0.9536
Epoch 8/20
3/3 [========

yhat = model.predict(x_valid)
label = decode_predictions(yhat)
label = label[0][0]
print('%s (%.2f%%)' % (label[1], label[2]*100))

### Evaluate

In [102]:
y_pred = model.predict(x_valid)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))

#Converting one hot encoded test label to label
test = list()
for i in range(len(labels_valid)):
    test.append(np.argmax(labels_valid[i]))

from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Accuracy is: 100.0


preds = model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])
np.corrcoef(preds, labels_valid)

In [36]:
cat_quantity = sum(labels_valid)

for i in range(1,10):
    print('threshold :'+str(.1*i))
    print(sum(labels_valid[preds > .1*i])/labels_valid[preds > .1*i].shape[0])

threshold :0.1
0.33774834437086093
threshold :0.2
0.33774834437086093
threshold :0.30000000000000004
0.33774834437086093
threshold :0.4
0.33774834437086093
threshold :0.5
0.33774834437086093
threshold :0.6000000000000001
0.33774834437086093
threshold :0.7000000000000001
0.33774834437086093
threshold :0.8
0.33774834437086093
threshold :0.9
0.33774834437086093


In [38]:
preds = model.predict(x_valid)
preds = np.asarray([pred[0] for pred in preds])
for i in range(1,10):
    t = .1*i
    print("{:.1f}:".format(t))
    tp = (preds > t)&(labels_valid==1)
    tn = (preds <= t)&(labels_valid==0)
    print(np.sum(np.where(tp|tn, 1, 0))/1024.)

0.1:
0.0498046875
0.2:
0.0498046875
0.3:
0.0498046875
0.4:
0.0498046875
0.5:
0.0498046875
0.6:
0.0498046875
0.7:
0.0498046875
0.8:
0.0498046875
0.9:
0.0498046875


In [37]:
print(preds.mean())
print(preds[labels_valid == 0].mean())
print(preds[labels_valid == 1].mean())

1.0
1.0
1.0


### 4.) Try a Different `base_model` 

* Repeat steps 3 and 4 above, only this time use a [pre-trained model other than VGG19](https://keras.io/applications/)
* (Answer in a markdown cell): Which model+layer works the best for this data? Why do you think that is?
* What makes for good positive examples? What makes for good negative examples? Why does the choice of negative examples matter?

In [121]:
import numpy as np
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model


base_model2 = InceptionV3(weights='imagenet',include_top=True)
xs,ys=299,299

for layer in base_model2.layers:
    print(layer.name)

input_4
conv2d_188
batch_normalization_188
activation_188
conv2d_189
batch_normalization_189
activation_189
conv2d_190
batch_normalization_190
activation_190
max_pooling2d_8
conv2d_191
batch_normalization_191
activation_191
conv2d_192
batch_normalization_192
activation_192
max_pooling2d_9
conv2d_196
batch_normalization_196
activation_196
conv2d_194
conv2d_197
batch_normalization_194
batch_normalization_197
activation_194
activation_197
average_pooling2d_18
conv2d_193
conv2d_195
conv2d_198
conv2d_199
batch_normalization_193
batch_normalization_195
batch_normalization_198
batch_normalization_199
activation_193
activation_195
activation_198
activation_199
mixed0
conv2d_203
batch_normalization_203
activation_203
conv2d_201
conv2d_204
batch_normalization_201
batch_normalization_204
activation_201
activation_204
average_pooling2d_19
conv2d_200
conv2d_202
conv2d_205
conv2d_206
batch_normalization_200
batch_normalization_202
batch_normalization_205
batch_normalization_206
activation_200
activa

### 2.) Determine your output layer



In [122]:
layer = 'predictions'

model2 = Model(inputs=base_model2.input, outputs=base_model2.get_layer(layer).output)

In [123]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 149, 149, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [124]:
model2 = Sequential()
model2.add(Dense(1024, input_dim=1000, activation='relu'))
model2.add(Dense(1024, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [125]:
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
dense_22 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 1025      
Total params: 2,075,649
Trainable params: 2,075,649
Non-trainable params: 0
_________________________________________________________________


In [126]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [127]:
history2 = model2.fit(x_train, labels_train, epochs=20, batch_size=64, validation_data=(x_valid, labels_valid))

Epoch 1/20
3/3 [==============================] - 1s 125ms/step - loss: 0.6772 - accuracy: 0.5835 - val_loss: 0.5996 - val_accuracy: 0.8013
Epoch 2/20
3/3 [==============================] - 0s 47ms/step - loss: 0.5691 - accuracy: 0.7892 - val_loss: 0.4940 - val_accuracy: 0.8146
Epoch 3/20
3/3 [==============================] - 0s 58ms/step - loss: 0.4489 - accuracy: 0.8174 - val_loss: 0.3828 - val_accuracy: 0.8278
Epoch 4/20
3/3 [==============================] - 0s 50ms/step - loss: 0.3311 - accuracy: 0.8385 - val_loss: 0.2820 - val_accuracy: 0.8874
Epoch 5/20
3/3 [==============================] - 0s 50ms/step - loss: 0.2295 - accuracy: 0.9245 - val_loss: 0.2015 - val_accuracy: 0.9603
Epoch 6/20
3/3 [==============================] - 0s 51ms/step - loss: 0.1500 - accuracy: 0.9764 - val_loss: 0.1474 - val_accuracy: 0.9669
Epoch 7/20
3/3 [==============================] - 0s 45ms/step - loss: 0.0831 - accuracy: 0.9967 - val_loss: 0.1165 - val_accuracy: 0.9669
Epoch 8/20
3/3 [==========

In [128]:
y_pred = model.predict(x_valid)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))

#Converting one hot encoded test label to label
test = list()
for i in range(len(labels_valid)):
    test.append(np.argmax(labels_valid[i]))

from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Accuracy is: 100.0


#### Which model+layer works the best for this data? Why do you think that is?
- I used InceptionV3 as my second model. In both cases the accuracy was 90%. However, when I used Sigmoid function instead of Softmax, the accuray level became way higher as Sigmoid is appropriate for binary data. 

#### What makes for good positive examples? What makes for good negative examples? Why does the choice of negative examples matter?
- I used images of chairs as postive examples. The more variation in chairs the better. Also, for negative examples, the diverse the images are the better. Thus, the classifier will know what not to tag as a positive image. 

### Practice Not Part of Assignment

In [102]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

SyntaxError: positional argument follows keyword argument (<ipython-input-102-59be4199dbaf>, line 2)

In [97]:
inputs =keras.Input(shape=(1000), name='ani_image')
x = layers.Flatten(name = 'flattened_img')(inputs)
#x=GlobalAveragePooling2D()(x)
x=layers.Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=layers.Dense(1024,activation='relu')(x) #dense layer 2
x=layers.Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='softmax')(x) #final layer with softmax activation

In [98]:
model=Model(inputs=inputs,outputs=preds)
model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ani_image (InputLayer)       [(None, 1000)]            0         
_________________________________________________________________
flattened_img (Flatten)      (None, 1000)              0         
_________________________________________________________________
dense_80 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
dense_81 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_82 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 513       
Total params: 2,599,937
Trainable params: 2,599,937
Non-trainable params: 0
________________________________________________

In [59]:
model = keras.Sequential([
    model.output,
    keras.Input(shape=(151)),
    layers.Dense(1024, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])

TypeError: The added layer must be an instance of class Layer. Found: KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_27/BiasAdd:0', description="created by layer 'dense_27'")

In [ ]:
optimizer='rmsprop', loss='categorical_crossentropy'

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'") at layer "block1_conv1". The following previous layers were accessed without issue: []

In [99]:
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [100]:
history = model.fit(x_train, 
                    labels_train,
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=10)

Epoch 1/10
5/5 [==============================] - 1s 22ms/step - loss: 0.0000e+00 - accuracy: 0.3413
Epoch 2/10
5/5 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 0.3539
Epoch 3/10
5/5 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - accuracy: 0.3170
Epoch 4/10
5/5 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - accuracy: 0.2949
Epoch 5/10
5/5 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - accuracy: 0.3027
Epoch 6/10
5/5 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 0.3222
Epoch 7/10
5/5 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accuracy: 0.3422
Epoch 8/10
5/5 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.3339
Epoch 9/10
5/5 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - accuracy: 0.3643
Epoch 10/10
5/5 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accura

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [69]:
inputs = Input(shape=(1000,151,1),name = 'image_input')

#Use the generated model 
output_model = model(inputs)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_model)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(8, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=inputs, outputs=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

ValueError: Input 0 is incompatible with layer sequential_1: expected shape=(None, 224, 224, 3), found shape=(None, 1000, 151, 1)

In [112]:
history = model.fit(x_train, 
                    labels_train,
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\sit

In [108]:


step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

NameError: name 'train_generator' is not defined

In [90]:
customAdam = keras.optimizers.Adam(lr=0.001)
my_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])


In [94]:
print('# Fit model on training data')
x_train_reshaped = x_train.reshape(x_train.shape[0], 50, 20, 1)
history = my_model.fit(x_train, 
                    labels_train,
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=10)

# Fit model on training data
Epoch 1/10


ValueError: in user code:

    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\alami\Anaconda3\envs\newenv\lib\sit

In [72]:
# Generate a model with all layers (with top)


#Add a layer where input is the output of the  second last layer 
x = Dense(8, activation='softmax', name='predictions')(model.output)

#Then create the corresponding model 
my_model = Model(input=model.input, output=x)
my_model.summary()

TypeError: ('Keyword argument not understood:', 'input')

In [42]:
SAMPLE_SIZE = xs, ys

SAMPLE_SIZE = img_size
img_size

1000

In [44]:
#total_pixels = img_size[0] *img_size[1] * 3
#total_pixels

In [71]:
#inputs = model.output
#inputs.reshape(n_images, 1000,1)

In [103]:
x=base_model.output
#x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [58]:
fc_size = 512

inputs = model.output
x=GlobalAveragePooling2D()(inputs)

x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='softmax')(x) #final layer with softmax activation

model = keras.Model(inputs=inputs, outputs=preds)

ValueError: Input 0 of layer global_average_pooling2d_7 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1000)

#transfer

#x=model.output
#x=GlobalAveragePooling2D()(x)

x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation


#another

#inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
#x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.


x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [55]:
inputs = model.output
x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.


x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: predictions/Softmax:0


ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "block1_conv1". The following previous layers were accessed without issue: []

In [ ]:
x = layers.Dense(fc_size, activation='relu', name='first_layer')(x_train)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

### Evaluate